# 355. Design Twitter

[leetcode](https://leetcode.com/problems/design-twitter/)

Design a simplified version of Twitter where users can post tweets, follow/unfollow another user, and is able to see the 10 most recent tweets in the user's news feed.

Implement the Twitter class:

Twitter() Initializes your twitter object.
void postTweet(int userId, int tweetId) Composes a new tweet with ID tweetId by the user userId. Each call to this function will be made with a unique tweetId.
List<Integer> getNewsFeed(int userId) Retrieves the 10 most recent tweet IDs in the user's news feed. Each item in the news feed must be posted by users who the user followed or by the user themself. Tweets must be ordered from most recent to least recent.
void follow(int followerId, int followeeId) The user with ID followerId started following the user with ID followeeId.
void unfollow(int followerId, int followeeId) The user with ID followerId started unfollowing the user with ID followeeId.

# Reasoning

[neetcodevideo](https://www.youtube.com/watch?v=pNichitDD2E&t=1s)

__NOTE__: this problem is similar to the "Merge K-sorted lists" problmem

The main problem is to figure out which data structureds to use for each function.  

Any user can _follow_ another user with differnt IDs. The simplest idea is to have a _hash map_ so that: 
- userID -> List of followeeID (each time aa new user follows, we extend the list)

To _unfollow_ it is not easy, as the removing from the list is a O(n) time operation. 

`!` A data structure that can be used add and remove in O(1), is the _hash set_.  -> 
- userID -> hash set of followeeID; than _follow_ and _unfollow_ can be realized with O(1) time. 


A user can post many tweets, so userID -> [tweetID], if we just use hash map, where we add the user to the end, this is O(1) time. So it ma work


The most complex function to implement, is the 'getNewsFeed(userID)' that must retreave 10 most recent tween IDs in the user's feed _from the list of the users that this user follows_ and that they need to be _ordered from the most recent to the least recent_. 

To get the 10 most recent we can just return 10 last tweets from the list. However, for many users, we need to "__merge these lists__" to get the most recent feeds. In order to do that we need to _keep track on the time_ that the tweet was posted. So we need to have lost of 'counts' (time) and associated tweetID, so we can merge the post histories to get the _ordered feed_. 

In Python we will be usin a `min heap` as there is not `max heep`.  

Overall, when a user requests a news feed, we 
1. Get the list of users this user follows (get the `has set` of users from our `hash map`)
2. For each followeeID, get the list of tweets (from the second `hash map`), where we get lists of _counts_ and _tweets_
3. Getting the combined, 10 most recent tweets, is equivalent to _merge K-sorted lists_ problem. The time complexity of this problem is O(10 * K), where K is the number of users, the user follows. A more efficient way to do this, is to set K pointers for the end of each user tweet lists and create a `min heap` out of them, and to get the minimum value. The minimum of the heap is O(log(K)) time, as since we would need to do it 10 times, the overall time coplxity is O(10 * log(K)).  
__NOTE__: however, creating the min heap, also requries time. Just pushing values into it is O(K*log(K)) or using `heapify` which O(k), so the overall time complexity would __still be O(k)__ which we originally had without heap. However, if we would need to get not 10 but N tweets, than heap would make the algorithm more efficient.  





In [9]:
from typing import List,Set
from collections import defaultdict
import heapq
class Twitter:

    def __init__(self):
        self.count = 0; # time
        self.tweetMap = defaultdict(list) # userID -> list[count,tweetIds]
        self.followMap = defaultdict(set) # userID -> set of followeeIDs
        

    def postTweet(self, userId: int, tweetId: int) -> None:
        self.tweetMap[userId].append([self.count,tweetId])
        self.count -= 1 # we decriment so we can use -min heap- in python

    def getNewsFeed(self, userId: int) -> List[int]:
        res = [] # ordered strarting from recent
        minHeap = []

        # add themselves to the map (so user gets his own posts)
        self.followMap[userId].add(userId)
        
        # create heat for the top layer
        for followeeID in self.followMap[userId]:
            if followeeID in self.tweetMap:    
                idx = len(self.tweetMap[followeeID])-1
                count, tweetID = self.tweetMap[followeeID][idx]
                # add followeeID so when we pop we know where to go next
                # add also idx - 1, the position we need to examine next
                minHeap.append([count,tweetID,followeeID,idx-1]) # count as a key (to get the)
        heapq.heapify(minHeap)
        # pop at most 10 values
        while minHeap and len(res) < 10:
            count,tweetID,followeeID,idx = heapq.heappop(minHeap)
            res.append(tweetID)
            if idx >= 0:
                # refill the minHeap with the next tweet/time from this user
                count,tweetID = self.tweetMap[followeeID][idx]
                heapq.heappush(minHeap,[count,tweetID,followeeID,idx-1])
        return res

    def follow(self, followerId: int, followeeId: int) -> None:
        self.followMap[followerId].add(followeeId)
        

    def unfollow(self, followerId: int, followeeId: int) -> None:
        if followeeId in self.followMap[followerId]:
            self.followMap[followerId].remove(followeeId)


# Your Twitter object will be instantiated and called as such:
# obj = Twitter()
# obj.postTweet(userId,tweetId)
# param_2 = obj.getNewsFeed(userId)
# obj.follow(followerId,followeeId)
# obj.unfollow(followerId,followeeId)
            
twitter = Twitter();
twitter.postTweet(1, 5); # User 1 posts a new tweet (id = 5).
twitter.getNewsFeed(1);  # User 1's news feed should return a list with 1 tweet id -> [5]. return [5]
twitter.follow(1, 2);    # User 1 follows user 2.
twitter.postTweet(2, 6); # User 2 posts a new tweet (id = 6).
twitter.getNewsFeed(1);  # User 1's news feed should return a list with 2 tweet ids -> [6, 5]. Tweet id 6 should precede tweet id 5 because it is posted after tweet id 5.
twitter.unfollow(1, 2);  # User 1 unfollows user 2.
twitter.getNewsFeed(1);  # User 1's news feed should return a list with 1 tweet id -> [5], since user 1 is no longer following user 2.
